In [1]:
import os, sys
sys.path.append('src')
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [5]:
pr_max_dict = pickle.load(open('data/tables_and_infos/pr_max_dict.pkl', 'rb'))
pr_max_dict

{np.int64(293): np.float64(19.0),
 np.int64(139): np.float64(21.0),
 np.int64(146): np.float64(27.0),
 np.int64(150): np.float64(13.0),
 np.int64(248): np.float64(21.0),
 np.int64(213): np.float64(12.0),
 np.int64(200): np.float64(14.0),
 np.int64(195): np.float64(11.0),
 np.int64(279): np.float64(39.0),
 np.int64(119): np.float64(47.0),
 np.int64(100): np.float64(47.0),
 np.int64(172): np.float64(47.0),
 np.int64(124): np.float64(12.0),
 np.int64(258): np.float64(11.0),
 np.int64(256): np.float64(9.0),
 np.int64(120): np.float64(6.0)}

In [2]:
from loader.data_loader_unbalanced_lump_short_list import load_session_data

# Define trial type dictionary
trial_type_dict = {}
i = 0
for name1 in ['left', 'right']:
    for name2 in ['hit', 'miss', 'ignore']:
        for name3 in ['', 'opto_early_4_', 'opto_late_4_', 'opto_early_5_', 'opto_late_5_', 'opto_early_6_', 'opto_late_6_']:
            trial_type_dict[name3 + name2 + '_' + name1] = i
            if 'opto' in name3:
                i += 1


ModuleNotFoundError: No module named 'nibabel'

In [ ]:

# Load a session
session_ind = 1  # or any session index
spike_data, behavior_data, area_ind_list, is_left, is_ALM, trial_type = load_session_data(session_ind, trial_type_dict)

In [ ]:
trial_type

In [ ]:
time_data = spike_data.mean(axis=0).mean(axis=-1)
plt.plot(time_data)

## Load IBL Data

Now let's load IBL (Neuropixels) data in a similar way

In [ ]:
from loader.data_loader_ibl import load_session_data as load_ibl_session_data

# Load an IBL session
# session_ind is the EID (experiment ID) string
# session_idx is the integer index for caching

# First, let's see what EIDs are available
with open("/u/jyao7/NeuroPaint/data/tables_and_infos/ibl_eids.txt") as file:
    ibl_eids = [line.rstrip() for line in file]

print(f"Total IBL sessions available: {len(ibl_eids)}")
print(f"First 5 EIDs: {ibl_eids[:5]}")

In [ ]:
# Load a specific IBL session
print(len(ibl_eids))
session_idx = 0  # Cache index
session_eid = ibl_eids[session_idx]  # Get the EID string

print(f"Loading session {session_idx}: {session_eid}")

# Load the IBL data
ibl_spike_data, ibl_behavior_data, ibl_area_ind_list, ibl_is_left, ibl_is_ALM, ibl_trial_type = load_ibl_session_data(
    session_eid, session_idx
)

print(f"\nIBL Data shapes:")
print(f"Spike data: {ibl_spike_data.shape} (N × T × K)")
print(f"Trial type (choice): {ibl_trial_type.shape}")
print(f"Number of neurons: {ibl_spike_data.shape[0]}")
print(f"Time bins per trial: {ibl_spike_data.shape[1]}")
print(f"Number of trials: {ibl_spike_data.shape[2]}")

In [ ]:
area_psths = {}
areaoi = ["PO", "LP", "DG", "CA1", "VISa", "VPM", "APN", "MRN"]
for area in areaoi:
    area_mask = [area in name for name in ibl_area_ind_list]
    area_spike_data = ibl_spike_data[area_mask, :, :]
    area_psth = area_spike_data
    area_psths[area] = area_psth

print(list(map(lambda x: (x, area_psths[x].shape), area_psths.keys())))

In [ ]:
plt.figure(figsize=(12, 6))
for area in areaoi:
    mean_psth = area_psths[area].mean(axis=0).mean(axis=-1)*100
    mean_psth -= mean_psth[:50].mean()
    plt.plot(mean_psth, label=area)
plt.legend()
plt.title("Average PSTH by Brain Area")
plt.xlabel("Time Bins")
plt.ylabel("Average Firing Rate")
plt.show()

In [ ]:
# Explore IBL brain regions
print(f"Unique brain regions in this session: {len(np.unique(ibl_area_ind_list))}")
print(f"Brain region indices: {np.unique(ibl_area_ind_list)}")

# Plot average firing rate over time (similar to MAP data)
ibl_time_data = ibl_spike_data.mean(axis=0).mean(axis=-1)
plt.figure(figsize=(10, 4))
plt.plot(ibl_time_data)
plt.xlabel('Time bins (10ms each)')
plt.ylabel('Average firing rate (spikes/bin)')
plt.title('IBL: Population firing rate over trial time')
plt.grid(True, alpha=0.3)
plt.show()